# 1. Importing libraries and creating folders

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.1.0
['.git', '.ipynb_checkpoints', 'models', 'model_name', 'Save Load and export models.ipynb', 'tmp', 'weights']


# 2. Importing dataset and Preprocessing

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 6s 1us/step


In [5]:
print('x_train shape: ',x_train.shape)
print('y_train shape: ',y_train.shape)
print('x_test shape: ',x_test.shape)
print('y_test shape: ',y_test.shape)

x_train shape:  (60000, 28, 28)
y_train shape:  (60000,)
x_test shape:  (10000, 28, 28)
y_test shape:  (10000,)


In [7]:
# NORMALIZING DATA

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1]*x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1]*x_test.shape[2]))

print('Updated Shape (train): ', x_train.shape)
print('Updated Shape (test): ', x_test.shape)

x_train = x_train/ 255.
x_test = x_test / 255.

Updated Shape (train):  (60000, 784)
Updated Shape (test):  (10000, 784)


In [8]:
# Categorical encoding

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [9]:
print('After catregorical (y_train)', y_train.shape)
print('After catregorical (y_test)', y_test.shape)

After catregorical (y_train) (60000, 10)
After catregorical (y_test) (10000, 10)


# 3. Creating Model

In [10]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


# 4. Saving weights

In [11]:
checkpoint_dir = 'weights/'    # as mentioned in folders varibale in first cell

In [12]:
_ = model.fit(x_train, y_train, 
              validation_data=(x_test, y_test),
             epochs = 2,
             batch_size= 512,
             callbacks= [
                 tf.keras.callbacks.ModelCheckpoint(
                     os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
                 monitor ='val_acc', save_weights_only = True, save_best_only = True
                 )
             ]
         )

# we could also save model architecture

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 65us/sample - loss: 0.5273 - acc: 0.8560 - val_loss: 0.2318 - val_acc: 0.9334
Epoch 2/2
60000/60000 [==============================] - 2s 39us/sample - loss: 0.1996 - acc: 0.9426 - val_loss: 0.1685 - val_acc: 0.9507


In [13]:
os.listdir(checkpoint_dir)

['checkpoint',
 'epoch_01_acc_0.9334.data-00000-of-00001',
 'epoch_01_acc_0.9334.index',
 'epoch_02_acc_0.9507.data-00000-of-00001',
 'epoch_02_acc_0.9507.index']

# 5. Loading weights

In [14]:
# without training our model

model = create_model()
model_eval = model.evaluate(x_test, y_test, verbose = False)

print('Loss: ', model_eval[0])
print('Accuracy: ', model_eval[1])

Loss:  2.329518714141846
Accuracy:  0.0652


In [17]:
# after loading our previously trained weights

model.load_weights('weights/epoch_02_acc_0.9507')      # caution: Dont use the .index extension in file name
model_eval = model.evaluate(x_test, y_test, verbose = False)

print('Loss: ', model_eval[0])
print('Accuracy: ', model_eval[1])

Loss:  0.16846339414417744
Accuracy:  0.9507


# 6. Saving Complete Model 